In [1]:
import database_bw
from bw_class import RFMProcessor
import mj_class

import warnings
warnings.filterwarnings('ignore')

c:\Users\pmj03\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
## 데이터베이스에서 데이터 불러오기(# database_bw 코드 이용 - 감사합니다..)
train = database_bw.making_dataframe_train_db('train_table')
test = database_bw.making_dataframe_test_db('test_table')

In [3]:
train.head(2)

,rowid,IND,고객ID,거래ID,거래날짜,제품ID,제품카테고리,수량,평균금액,배송료,쿠폰상태,월,성별,고객지역,가입기간,쿠폰코드,할인율,GST
0,1,0,USER_1358,Transaction_0000,2019-01-01,Product_0981,Nest-USA,1,153.71,6.5,Used,1,남,Chicago,12,ELEC10,10.0,0.1
1,2,1,USER_1358,Transaction_0001,2019-01-01,Product_0981,Nest-USA,1,153.71,6.5,Used,1,남,Chicago,12,ELEC10,10.0,0.1


In [4]:
# 전처리
train =  mj_class.mj_preprocessing(train)
train.apply_my_function()
train = train.return_dataframe()

In [5]:
train.head()

,고객ID,거래ID,거래날짜,제품ID,제품카테고리,수량,평균금액,배송료,쿠폰상태,월,성별,고객지역,가입기간,쿠폰코드,할인율,GST,고객소비액,매출
0,USER_1358,Transaction_0000,2019-01-01,Product_0981,Nest-USA,1,153.71,6.5,Used,1,남,Chicago,12,ELEC10,10.0,0.10,158.6729,153.71
1,USER_1358,Transaction_0001,2019-01-01,Product_0981,Nest-USA,1,153.71,6.5,Used,1,남,Chicago,12,ELEC10,10.0,0.10,158.6729,153.71
2,USER_1358,Transaction_0002,2019-01-01,Product_0904,Office,1,2.05,6.5,Used,1,남,Chicago,12,OFF10,10.0,0.10,8.5295,2.05
3,USER_1358,Transaction_0003,2019-01-01,Product_0203,Apparel,5,17.53,6.5,Not Used,1,남,Chicago,12,SALE10,0.0,0.18,109.9270,87.65
4,USER_1358,Transaction_0003,2019-01-01,Product_0848,Bags,1,16.50,6.5,Used,1,남,Chicago,12,AIO10,10.0,0.18,24.0230,16.50


In [6]:
train.columns

Index(['고객ID', '거래ID', '거래날짜', '제품ID', '제품카테고리', '수량', '평균금액', '배송료', '쿠폰상태',
       '월', '성별', '고객지역', '가입기간', '쿠폰코드', '할인율', 'GST', '고객소비액', '매출'],
      dtype='object')

In [7]:
# rfm분석 한 dataframe
processor = RFMProcessor(train) 
rfm_without_outliers, rfm_outliers, rfm_without_outliers_log, X_scaled = processor.process_data()
processor.fit_clustering(X_scaled, n_clusters=4)
rfm = processor.predict(train)

In [8]:
rfm.head()

,고객ID,Recency,Frequency,Monetary,Recency_log,Frequency_log,Monetary_log,Cluster
0,USER_0000,47,1,174.98174,3.850148,0.000000,5.164682,2.0
1,USER_0001,222,13,12901.28216,5.402677,2.564949,9.465082,-1.0
2,USER_0002,13,8,1689.55594,2.564949,2.079442,7.432221,3.0
3,USER_0004,47,13,2007.42900,3.850148,2.564949,7.604610,1.0
4,USER_0005,70,3,784.03320,4.248495,1.098612,6.664451,0.0


In [9]:
rfm.columns

Index(['고객ID', 'Recency', 'Frequency', 'Monetary', 'Recency_log',
       'Frequency_log', 'Monetary_log', 'Cluster'],
      dtype='object')

In [10]:
rfm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1294 entries, 0 to 1293
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   고객ID           1294 non-null   object 
 1   Recency        1294 non-null   int64  
 2   Frequency      1294 non-null   int64  
 3   Monetary       1294 non-null   float64
 4   Recency_log    1294 non-null   float64
 5   Frequency_log  1294 non-null   float64
 6   Monetary_log   1294 non-null   float64
 7   Cluster        1294 non-null   float64
dtypes: float64(5), int64(2), object(1)
memory usage: 81.0+ KB


In [11]:
viz = mj_class.mj_visualization(train, rfm)

In [12]:
viz.cluster_calculate_and_plot_arppu()
viz.month_calculate_and_plot_arppu()
viz.area_calculate_and_plot_arppu()
viz.calculate_and_plot_arppu_by_subscription_period_grouped()
viz.area_calculate_and_plot_mapbox()

In [13]:
apr = mj_class.mj_apriori(min_support=0.6, min_confidence=0.005, min_lift=1, top_n=5)

In [14]:
apr.apriori_analysis(train)

Processing 9 combinations | Sampling itemset size 32
지지도: 두 제품을 모두 구매한 고객 수의 비율
신뢰도: A를 구매한 고객 중 B를 구매한 고객의 비율
향상도: 마케팅 효과 증가율


,제품,지지도,신뢰도,향상도
0,Apparel => Office,0.65,0.83,1.11
1,Office => Apparel,0.65,0.87,1.11
2,Apparel => Nest-USA,0.65,0.92,1.10
3,Nest-USA => Apparel,0.65,0.78,1.10
4,Office => Nest-USA,0.68,0.90,1.07
